In [1]:
import os
import numpy as np
import torch 
import torchvision
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import utils
import torch.nn as nn
import torch.nn.functional as nn
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

In [2]:
#Inpyt image size
ISIZE = (800, 800)

# Imagenet statistics
imagenet_stats = np.array([[0.485, 0.456, 0.406] , [0.229, 0.224, 0.225]])


In [3]:
# helper functions

def normalize(im):
    # convert image to float 
    im = im / 255.
    """ Normalize with image net stats"""
    return (im - imagenet_stats[0])/imagenet_stats[1]

def train_val_dataset(dataset, val_split = 0.1):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size = val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val']   = Subset(dataset, val_idx)
    return datasets

In [4]:
# load data set and tranforms

class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all the image files, sorting them to ensure they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        
    def __getitem__(self, idx):
        # load image and masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks" , self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        img = img.resize(ISIZE)
        img = np.array(img)
        img = normalize(img)
        img = img.transpose(2,0,1)
        img = torch.as_tensor(img, dtype = torch.float32)
        
        mask = Image.open(mask_path)
        mask = mask.resize(ISIZE)
        mask = np.array(mask)
        obj_ids = np.unique(mask)            # instances are encoded as different colors (0--backhroung)
        obj_ids = obj_ids[1:]                # first id is background remove it
        # split the color-encoded mask into a set of binary masks (i.e true or false)
        masks = mask == obj_ids[:, None, None]
        
        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []

        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
            
        # convert to torch tensor
        boxes = torch.as_tensor(boxes, dtype = torch.float32)   # box dims
        masks = torch.as_tensor(masks, dtype=torch.uint8)       # true or false
        labels = torch.ones((num_objs,) , dtype = torch.int64)  # no od persons

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])   # area
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [5]:
dataset = PennFudanDataset('./data/PennFudanPed/', None)

datasets = train_val_dataset(dataset)
print(len(datasets['train']))
print(len(datasets['val']))

data_loader = torch.utils.data.DataLoader(datasets['train'], batch_size = 15, shuffle = True)
data_loader_val = torch.utils.data.DataLoader(datasets['val'], batch_size=15, shuffle=True)

153
17


In [10]:
is_cuda= False
if torch.cuda.is_available():
    is_cuda = True
print(is_cuda)

model = torchvision.models.vgg16(pretrained = True)
if is_cuda:
    model.cuda()
model.eval()

for param in model.features.parameters():
    param.requires_grad = False
    
fe = list(model.features)
req_features = []
for j, i in enumerate(fe[0:30]):
    req_features.append(i)                          # Remove last pooling layer
    print(i)
    
#print(req_features)

False
Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplace=True)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
ReLU(inplac

In [ ]:
def train_epochs(req_features, model, optimizer, train_dl, val_dl, epochs = 10, rpn_lambda = 10):
    for epoch in range(epochs):
        model.train()   # https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch
        total = 0
        sum_loss = 0
        sum_loss_cls = 0
        sum_loss_loc = 0
        idx = 0
        for images, targets in train_dl:
            idx += 1
            num_batch = len(images)
            print(f'batch --> {idx} , epoch --> {epoch}')
            imgs_torch_all = torch.stack([item for item in images])
            imgs_clone     = imgs_torch_all.clone()   # copies to a new tensor
            
            for feature in req_features:
                imgs_clone = feature(imgs_clone    )
            
         

In [ ]:
class RPN(nn.Module):
    
    def __init__(self, in_channels = 512, mid_channels = 512, n_anchor = 9):
        super(RNN, self).__init__()
        self.in_channels  = in_channels   # depends on output of feature map in vgg 16 its 512
        self.mid_channels = mid_channels
        self.n_anchor     = n_anchor      # no of anchors in each loacation
        
        self.conv1 = nn.Conv2d(self.in_channels, self.mid_channels, kernel_size = 3, stride = 1, padding =1)
        self.reg_layer = nn.Conv2d(self.mid_channels, n_anchor * 4 , 1 , 1 , 0)
        self.cls_layer = nn.Conv2d(self.mid_channels, n_anchor * 2 , 1 , 1 , 0)        # see the paper
        
        
        # Conv sliding layer
        self.conv1.weight.data.normal_(0,0.01)
        self.conv1.bias.data.zero_()
        # Regression Layer
        self.ref_layer.weight.data.normal_(0,0.01)
        self.ref_layer.bias.data.zero_()
        # Classification Layer = RPN
        self.cls_layer.weight.data.normal_(0,0.01)
        self.cls_layer.bias.data.zero_()
            
    def forward(self, k): 
        bat_num = k.shape[0]
        x = self.conv1(k)
        pred_anchor_boxes = self.reg_layer(x)
        pred_cls_scores   = self.cls_layer(x)

In [ ]:
learning_rate = 0.0015
num_epochs    = 20
model = RPN()

parameters = model.parameters()
optimizer = torch.optim.Adam(parameters , lr = learning_rate)

In [ ]:
model = train_epochs(req_features, model, optimizer, data_loader, None, epochs = num_epochs)

In [8]:
# About dilation max pooling https://datascience.stackexchange.com/questions/28881/what-is-dilated-pooling-and-how-it-works-mathematically